# INIT 

In [1]:
#import necessories

import pandas as pd
import numpy as np
from datetime import datetime


# Data Wrangling

In [2]:
# Data Wrangling:- Removing Nan values,droping NAN value column

rfm=(
    pd
    .read_csv(r'E:\1 Work\GUI\transaction data.csv',parse_dates=['Date'])
    .drop(['COUPON_CODE','DISCOUNT','ID','ONLINE_OFFLINE_PURCHASE','PRODUCT_ID','QUANTITY','Retuned/Purchased','RETURNED','SIZE'],axis=1)
    .dropna()
)

# Feature Engineering

In [3]:
#Rename columns for better analysis

rfm.rename(columns={'CUSTOMER_ID':'Customer ID','TOTAL_BILL_PRICE':'Total Bill price','TRANSACTION_ID':'Transaction_ID'},inplace=True)

In [4]:
#featured dataframe
rfm

,Customer ID,Date,Total Bill price,Transaction_ID
4531,1000044.0,2018-10-31,1799.0,6753985
4532,1000044.0,2019-12-26,2999.0,10011927
4533,1000044.0,2020-01-17,1199.0,10031470
4534,1000044.0,2020-04-22,2599.0,10066565
4535,1000074.0,2018-10-31,1899.0,6754151
...,...,...,...,...
29994,1075293.0,2019-01-30,279.0,7496914
29995,1075293.0,2019-01-30,449.0,7496913
29996,1075326.0,2019-01-30,0.0,7497178
29997,1075326.0,2020-05-22,1499.0,10085100


# RFM Analysis

In [5]:
#RFM Of Dataset

max_date = max(rfm['Date'])
print (type(max_date))

rfm_isr = rfm.groupby('Customer ID').agg({'Date': lambda x:(max_date-max(x)).days,
                     'Transaction_ID':'count', 
                     'Total Bill price':'sum'})
rfm_isr.rename(columns = {'Date':'Recency',
                         'Transaction_ID': 'Frequency',
                         'Total Bill price': 'Monetary'},inplace = True)
r_labels = range(5,0,-1)
r_groups = pd.qcut(rfm_isr.Recency,q=5,labels = r_labels)
f_labels = range(1,6)
f_groups = pd.cut(rfm_isr.Frequency,labels = f_labels, bins = 5)
m_labels = range(1,6)
m_groups = pd.qcut(rfm_isr.Monetary,q=5,labels = m_labels)
rfm_isr['R'] = r_groups.values
rfm_isr['F'] = f_groups.values
rfm_isr['M'] = m_groups.values

#Converting Datatypes from Categorical to Float
rfm_isr['R']=rfm_isr.R.astype(float)
rfm_isr['F']=rfm_isr.F.astype(float)
rfm_isr['M']=rfm_isr.M.astype(float)

# Standerdizing the columns 'R','F','M':-
rfm_isr['R']=(rfm_isr['R']/rfm_isr['R'].max())*100
rfm_isr['F']=(rfm_isr['F']/rfm_isr['F'].max())*100
rfm_isr['M']=(rfm_isr['M']/rfm_isr['M'].max())*100

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [6]:
#Function To Calculate RFM based on input given by user for R,F and M individually
def RFM_score(df: pd.DataFrame) -> pd.DataFrame:
    # CopyData
    _df = df.copy(deep=True)
    
    # Ask For weights
    r=float(input('Enter weightage for Recency:- ')) or 1/3
    f=float(input('Enter weightage for Frequency:- ')) or 1/3
    m=float(input('Enter weightage for Monitary:- ')) or 1/3
    
    # # Assert weights to be positive
    if (1<=r<=0) and (1<=f<=0) and (1<=m<=0):
        raise ValueError("The RFM weights should be positive and between One and Zero.")
    
    # Check Weights
    if r+f+m==1:
        _df['RFM_score'] = (_df['R']*r + _df['F']*f + _df['M']*m).astype(int)
    elif r+f+m <= 1:
        m = 1 - r - f
        _df['RFM_score'] = (_df['R']*r + _df['F']*f + _df['M']*m).astype(int)
    else:
        raise ValueError('Wrong weightage for RFM, all weights should add up to 1.')
    
    return _df

In [7]:
rfm_isr=RFM_score(rfm_isr)

In [8]:
#Customer Segmentation Based On Value of Customer
rfm_labels = range(5,0,-1)
rfm_groups = pd.qcut(rfm_isr['RFM_score'],q=5,labels = rfm_labels)
rfm_isr['RFM_grp']=rfm_groups.values


In [9]:
def name(row):
    if row == 5:
        return 'High Value Customer'
    elif row == 4:
        return 'Best Customer'
    elif row == 3:
        return 'Mid Value Customer'
    elif row == 2:
        return 'Low Value Customer'
    else:
        return 'Lost Customer'

rfm_isr['Customer Segmentation']=rfm_isr['RFM_grp'].apply(name)

# Customer Segmentation based on their activity behaviour

In [10]:
rfm_isr['RFM_score']=rfm_isr['RFM_score'].astype(int)
rfm_isr['R']=rfm_isr['R'].astype(int)
rfm_isr['F']=rfm_isr['F'].astype(int)
rfm_isr['M']=rfm_isr['M'].astype(int)
rfm_isr.reset_index(inplace=True)

In [11]:
rfm_isr

,Customer ID,Recency,Frequency,Monetary,R,F,M,RFM_score,RFM_grp,Customer Segmentation
0,1000044.0,769,4,8596.0,80,20,100,66,2,Low Value Customer
1,1000074.0,1308,1,1899.0,20,20,40,24,5,High Value Customer
2,1000109.0,632,2,1898.0,80,20,40,54,2,Low Value Customer
3,1000123.0,263,6,5294.0,100,20,80,72,1,Lost Customer
4,1000125.0,1308,1,1599.0,20,20,40,24,5,High Value Customer
...,...,...,...,...,...,...,...,...,...,...
5961,1075268.0,1217,1,3299.0,60,20,80,52,3,Mid Value Customer
5962,1075270.0,624,3,2497.0,80,20,60,58,2,Low Value Customer
5963,1075271.0,1217,1,1199.0,60,20,40,44,3,Mid Value Customer
5964,1075293.0,1217,2,728.0,60,20,20,40,4,Best Customer


In [12]:
rfm_seg=rfm_isr
rfm_seg.sort_values(by='Customer Segmentation')

,Customer ID,Recency,Frequency,Monetary,R,F,M,RFM_score,RFM_grp,Customer Segmentation
4066,1055507.0,1253,1,999.0,40,20,20,30,5,High Value Customer
4965,1063982.0,1234,1,1899.0,40,20,40,34,5,High Value Customer
4966,1063984.0,1234,1,999.0,40,20,20,30,5,High Value Customer
1941,1027744.0,1285,1,1299.0,40,20,40,34,5,High Value Customer
1937,1027726.0,1285,1,1899.0,40,20,40,34,5,High Value Customer
...,...,...,...,...,...,...,...,...,...,...
1558,1025466.0,189,13,8189.0,100,20,100,76,1,Lost Customer
5120,1065880.0,357,14,7036.0,100,20,100,76,1,Lost Customer
3297,1041381.0,74,12,10939.0,100,20,100,76,1,Lost Customer
1583,1025623.0,21,6,4900.0,100,20,80,72,1,Lost Customer
